In [1]:
# Core libraries for JSON/SQL/CSV analysis
import json
import sys
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path

# Add project root to path (for running from notebooks/ or project root)
_root = Path.cwd().parent if (Path.cwd().parent / "data" / "raw").exists() else Path.cwd()
if str(_root) not in sys.path:
    sys.path.insert(0, str(_root))
from config import DATA_RAW, DATA_PROCESSED, DB_PATH

In [2]:
# Quick inventory of NDJSON files and basic shape
ndjson_files = sorted(DATA_RAW.glob("*.ndjson"))

print(f"Found {len(ndjson_files)} ndjson files")
for path in ndjson_files:
    print("-", path.name)

# Load each ndjson file into a dataframe
dfs = {}
for path in ndjson_files:
    df = pd.read_json(path, lines=True)
    dfs[path.name] = df

# Print head of each dataframe
for name, df in dfs.items():
    print(f"\n{name} rows: {len(df)} cols: {len(df.columns)}")

Found 12 ndjson files
- AllergyIntolerance.000.ndjson
- Condition.000.ndjson
- Device.000.ndjson
- DiagnosticReport.000.ndjson
- DocumentReference.000.ndjson
- Encounter.000.ndjson
- Immunization.000.ndjson
- MedicationRequest.000.ndjson
- Observation.000.ndjson
- Observation.001.ndjson
- Patient.000.ndjson
- Procedure.000.ndjson

AllergyIntolerance.000.ndjson rows: 75 cols: 12

Condition.000.ndjson rows: 4559 cols: 12

Device.000.ndjson rows: 208 cols: 13

DiagnosticReport.000.ndjson rows: 15747 cols: 13

DocumentReference.000.ndjson rows: 9069 cols: 13

Encounter.000.ndjson rows: 9069 cols: 14

Immunization.000.ndjson rows: 1818 cols: 10

MedicationRequest.000.ndjson rows: 10367 cols: 14

Observation.000.ndjson rows: 52820 cols: 14

Observation.001.ndjson rows: 12727 cols: 14

Patient.000.ndjson rows: 120 cols: 16

Procedure.000.ndjson rows: 14085 cols: 11


In [3]:
# Flatten NDJSON into SQLite tables for SQL queries
ndjson_files = sorted(DATA_RAW.glob("*.ndjson"))

# Flatten nested dict/list structures
def _flatten_value(value, prefix, out):
    if isinstance(value, dict):
        for k, v in value.items():
            _flatten_value(v, f"{prefix}.{k}" if prefix else k, out)
    elif isinstance(value, list):
        for i, v in enumerate(value):
            _flatten_value(v, f"{prefix}[{i}]", out)
        if len(value) == 0:
            out[prefix] = None
    else:
        out[prefix] = value

def flatten_record(record):
    out = {}
    _flatten_value(record, "", out)
    return out

# Load NDJSON files into SQLite
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
conn = sqlite3.connect(DB_PATH)
name_counts = {}

for path in ndjson_files:
    with path.open("r", encoding="utf-8") as f:
        records = [flatten_record(json.loads(line)) for line in f if line.strip()]

    df = pd.DataFrame(records)

    base_name = path.name.split(".", 1)[0].lower() ## normalize file names
    count = name_counts.get(base_name, 0)
    table_name = f"{base_name}_{count}" if count > 0 else base_name ## create unique table names for duplicate file names
    name_counts[base_name] = count + 1 ## increment count for next table

    df.to_sql(table_name, conn, if_exists="replace", index=False)
    print(f"Loaded {len(df)} rows into '{table_name}'")

# Combine observation files into one union table
obs_cols_query = "PRAGMA table_info(observation);"
obs_cols = [row[1] for row in conn.execute(obs_cols_query).fetchall()]

obs1_cols_query = "PRAGMA table_info(observation_1);"
obs1_cols = [row[1] for row in conn.execute(obs1_cols_query).fetchall()]

common_cols = [col for col in obs_cols if col in obs1_cols]

union_cols_str = ", ".join([f'"{col}"' for col in common_cols])

union_table_sql = f"""
CREATE TABLE IF NOT EXISTS observation_union AS
SELECT * FROM observation
UNION ALL
SELECT * FROM observation_1;
"""
conn.execute(union_table_sql)
conn.commit()
print(f"Created 'observation_union' with {len(common_cols)} columns")

conn.close()

Loaded 75 rows into 'allergyintolerance'
Loaded 4559 rows into 'condition'
Loaded 208 rows into 'device'
Loaded 15747 rows into 'diagnosticreport'
Loaded 9069 rows into 'documentreference'
Loaded 9069 rows into 'encounter'
Loaded 1818 rows into 'immunization'
Loaded 10367 rows into 'medicationrequest'
Loaded 52820 rows into 'observation'
Loaded 12727 rows into 'observation_1'
Loaded 120 rows into 'patient'
Loaded 14085 rows into 'procedure'
Created 'observation_union' with 237 columns


In [4]:
# Inspect exported CSV schemas for Tableau
csv_files = sorted(DATA_PROCESSED.glob("*.csv"))

columns_map = {}
for path in csv_files:
    df = pd.read_csv(path, nrows=1)
    columns_map[path.name] = list(df.columns)

for name, cols in columns_map.items():
    print(f"\n{name} columns ({len(cols)}):")
    print(cols)


allergyintolerance.csv columns (11):
['patient_id', 'allergy_type', 'allergy_category', 'allergy_date', 'allergy_name', 'allergy_reaction_1', 'allergy_reaction_2', 'allergy_reaction_3', 'allergy_reaction_4', 'allergy_reaction_5', 'allergy_reaction_6']

condition.csv columns (7):
['patient_id', 'encounter_id', 'condition_id', 'condition', 'condition_resolved', 'condition_onset_date', 'condition_abatement_date']

device.csv columns (4):
['patient_id', 'device_expiration_date', 'device_name', 'device_expired']

encounter.csv columns (7):
['patient_id', 'encounter_id', 'encounter_type', 'encounter_start_date', 'encounter_end_date', 'encounter_length_hours', 'encounter_reason']

immunization.csv columns (4):
['patient_id', 'encounter_id', 'vaccine_name', 'vaccine_date']

medicationrequest.csv columns (4):
['patient_id', 'encounter_id', 'status', 'medication_name']

observation.csv columns (48):
['patient_id', 'encounter_id', 'observation_date', 'observation_category', 'observation_name', '